# Chapter 6

# Section 6.3.2 Subsample Analysis

# VAR-BEKK-GARCH model for Period 3 (2015/12 - 2020/1) (Section 6.3.2)

In [1]:
library(xts)
library(BEKKs)
library(MTS)
library(vars)
library(stargazer)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘MTS’


The following object is masked from ‘package:BEKKs’:

    backtest


Loading required package: MASS

Loading required package: strucchange

Loading required package: sandwich

Loading required package: urca

Loading required package: lmtest


Attaching package: ‘vars’


The following object is masked from ‘package:MTS’:

    VAR



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




# Dataset Preperation

In [2]:
fr_d <- readRDS("fr_d.Rds")
fp_d <- readRDS("fp_d.Rds")
cr_d <- readRDS("cr_d.Rds")
cp_d <- readRDS("cp_d.Rds")
head(cr_d)
head(fr_d)
us_d <- cr_d[,1]
l_d <- cr_d[,2]

                     usc           lc
2007-07-06  0.0033026687  0.005272420
2007-07-09 -0.0108928380 -0.013233541
2007-07-10 -0.0378502241 -0.040784991
2007-07-11  0.0210431058  0.019240172
2007-07-12  0.0057943185  0.004526943
2007-07-13 -0.0009633912 -0.004526943

                  eurusd        gbpusd        chfusd        usdgbp       eurgbp
2007-07-06  2.240547e-03 -0.0004972527 -0.0003044418  0.0007037652  0.002659380
2007-07-09  7.337565e-05  0.0022605055  0.0012172115 -0.0025157246 -0.001993871
2007-07-10  8.584371e-03  0.0057647029  0.0115503355 -0.0056582954  0.002805051
2007-07-11 -7.274844e-05  0.0027838359 -0.0029504763 -0.0026379885 -0.002952903
2007-07-12  3.485969e-03 -0.0009845428  0.0028903495  0.0008124302  0.004573291
2007-07-13 -7.252421e-04  0.0019435396  0.0000000000 -0.0020323145 -0.003391083
                  chfgbp        ghsusd        ghsgbp
2007-07-06  0.0001273483 -0.0033264938 -2.614079e-03
2007-07-09 -0.0009873737  0.0040086898  1.236683e-03
2007-07-10  0.0059364761 -0.0002705906 -5.801912e-03
2007-07-11 -0.0057330414  0.0026887042  4.994937e-05
2007-07-12  0.0037670817  0.0027212691  3.716576e-03
2007-07-13 -0.0020261005 -0.0051426368 -7.210559e-03

In [3]:
d <- na.omit(merge(cr_d,fr_d))
colnames(d) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")

In [4]:
d_p <- na.omit(merge(cp_d,fp_d))
colnames(d_p) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")

# Period 3 2015/12 - 2020/1 Data

In [5]:
d <- d["2015-12/2020-1"]
str(d)
head(d)
tail(d)

An xts object on 2015-12-01 / 2020-01-31 containing: 
  Data:    double [1039, 10]
  Columns: US Cocoa, London Cocoa, EURUSD, GBPUSD, CHFUSD ... with 5 more columns
  Index:   Date [1039] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:216] 37 42 100 101 122 123 124 128 142 162 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                US Cocoa London Cocoa       EURUSD       GBPUSD       CHFUSD
2015-12-01 -0.0005991612 -0.000877193  0.006321969  0.001625882  0.002722348
2015-12-02 -0.0011994004  0.007431728 -0.001553197 -0.008624044  0.007359363
2015-12-03  0.0157764844  0.005646052  0.030294804  0.012891829  0.024842875
2015-12-04  0.0011806377  0.001730853 -0.005039410 -0.002181242 -0.003532781
2015-12-07  0.0079330517  0.005604671 -0.004695709 -0.003944909 -0.003295225
2015-12-08 -0.0300037359 -0.019536111  0.005338500 -0.002993915  0.008267391
                 USDGBP        EURGBP        CHFGBP        GHSUSD        GHSGBP
2015-12-01 -0.001732907  4.759377e-03  0.0010835914 -0.0009211267 -0.0025456561
2015-12-02  0.008709418  6.216905e-03  0.0160384054  0.0012889628  0.0099212308
2015-12-03 -0.012789822  1.717679e-02  0.0119562529 -0.0073730523 -0.0201777897
2015-12-04  0.002042284 -2.772965e-03 -0.0014302385  0.0113934382  0.0134894260
2015-12-07  0.003996838 -6.942275e-05  0.0006777364 -0.004388

                US Cocoa London Cocoa        EURUSD        GBPUSD        CHFUSD
2020-01-24 -0.0115358165 -0.004590674 -0.0026269321 -0.0034732197 -0.0023766229
2020-01-27  0.0003625159  0.001532567 -0.0006351223 -0.0017219941  0.0016496850
2020-01-28 -0.0105666860 -0.004092077  0.0003629764 -0.0020703147 -0.0034966769
2020-01-29  0.0087527911  0.005112486 -0.0009985023 -0.0005374487 -0.0002432794
2020-01-30  0.0272223113  0.019692633  0.0019053674  0.0057052210  0.0038369119
2020-01-31 -0.0103002154 -0.015113638  0.0056042811  0.0080239246  0.0065228652
                  USDGBP        EURGBP        CHFGBP        GHSUSD       GHSGBP
2020-01-24  0.0035370444  0.0010679325  0.0010801539 -1.018738e-03  0.002464379
2020-01-27  0.0016985697  0.0009483168  0.0033600691  5.026346e-05  0.001733125
2020-01-28  0.0020214542  0.0024850613 -0.0013933754  2.312178e-03  0.004415901
2020-01-29  0.0005210029 -0.0005911214  0.0003168467  2.848696e-03  0.003384184
2020-01-30 -0.0056805350 -0.0035540850 -

# VAR-BEKK Models for the subsample

#### The estimation of the VAR-BEKK model is conducted in two stages, VAR model is the model specification for the mean equation (first stage) and BEKK is the specification for the variance equation (second stage).

## (i) Spillover between US Cocoa and London Cocoa

In [6]:
cr_d <- cr_d["2015-12/2020-1"]
str(cr_d)

An xts object on 2015-12-01 / 2020-01-31 containing: 
  Data:    double [1039, 2]
  Columns: usc, lc
  Index:   Date [1039] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:159] 37 42 100 101 122 123 140 160 184 214 ...
  ..- attr(*, "index")= num [1:159] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


In [7]:
VARselect(cr_d)

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.809433e+01,-1.808863e+01,-1.808520e+01,-1.808334e+01,-1.808093e+01,-1.807903e+01,-1.807539e+01,-1.807030e+01,-1.806392e+01,-1.805900e+01
HQ(n),-1.808340e+01,-1.807043e+01,-1.805971e+01,-1.805056e+01,-1.804087e+01,-1.803169e+01,-1.802077e+01,-1.800839e+01,-1.799473e+01,-1.798253e+01
SC(n),-1.806554e+01,-1.804066e+01,-1.801804e+01,-1.799699e+01,-1.797539e+01,-1.795430e+01,-1.793147e+01,-1.790719e+01,-1.788163e+01,-1.785751e+01
FPE(n),1.385905e-08,1.393820e-08,1.398610e-08,1.401224e-08,1.404601e-08,1.407269e-08,1.412409e-08,1.419619e-08,1.428702e-08,1.435759e-08


In [8]:
var_cr<-VAR(cr_d,p = 1,type = "const")
summary(var_cr)
res_cr <- residuals(var_cr)
res_cr <- as.data.frame(res_cr)


VAR Estimation Results:
Endogenous variables: usc, lc 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6455.758 
Roots of the characteristic polynomial:
0.04301 0.006106
Call:
VAR(y = cr_d, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + lc.l1 + const 

         Estimate Std. Error t value Pr(>|t|)
usc.l1 -0.0682652  0.0646708  -1.056    0.291
lc.l1   0.0507101  0.0770207   0.658    0.510
const  -0.0002048  0.0005282  -0.388    0.698


Residual standard error: 0.01701 on 1035 degrees of freedom
Multiple R-Squared: 0.001374,	Adjusted R-squared: -0.0005557 
F-statistic: 0.712 on 2 and 1035 DF,  p-value: 0.4909 


Estimation results for equation lc: 
lc = usc.l1 + lc.l1 + const 

         Estimate Std. Error t value Pr(>|t|)
usc.l1 -0.0370330  0.0543444  -0.681    0.496
lc.l1   0.0313572  0.0647224   0.484    0.628
const  -0.0001333  0.0004438  -0.300    0.764


Residual standard error: 0.0143 on 1035 degrees of freedom
Multiple R-Squared:

In [9]:
ind_var <- index(cr_d)[-1]
str(ind_var)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [10]:
res_cr <- xts(res_cr,order.by = ind_var)
str(res_cr)
head(res_cr)

An xts object on 2015-12-02 / 2020-01-31 containing: 
  Data:    double [1038, 2]
  Columns: usc, lc
  Index:   Date [1038] (TZ: "UTC")


                    usc           lc
2015-12-02 -0.000990973  0.007570316
2015-12-03  0.015522590  0.005501867
2015-12-04  0.002176158  0.002271329
2015-12-07  0.008130723  0.005727389
2015-12-08 -0.029541551 -0.019284803
2015-12-09  0.015597512  0.004881802

In [11]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk_fit <- bekk_fit(spec = bekk_spec,data = res_cr)

In [12]:
bekk_fit$A
bekk_fit$A_t
bekk_fit$G
bekk_fit$G_t

0.10911555,-0.1914859
0.07974417,0.3807527


1.3379129,-2.721746
0.8879919,4.756689


1.01635070,0.1480469
-0.04622508,0.8098921


21.1745957,3.197701
-0.8642078,15.914779


### VAR-BEKK Model Table for US Cocoa - London Cocoa

In [13]:
bekkc <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_cr)$varresult$usc$coefficients["const","Estimate"],summary(var_cr)$varresult$lc$coefficients["const","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Estimate"],
                                   bekk_fit$A[1,1],bekk_fit$A[1,2],bekk_fit$A[2,1],bekk_fit$A[2,2],bekk_fit$G[1,1],bekk_fit$G[1,2],bekk_fit$G[2,1],bekk_fit$G[2,2]),
                   Std_error = c(summary(var_cr)$varresult$usc$coefficients["const","Std. Error"],summary(var_cr)$varresult$lc$coefficients["const","Std. Error"],
                               summary(var_cr)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Std. Error"],
                                 summary(var_cr)$varresult$lc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Std. Error"],
                                 bekk_fit$A_sd[1,1],bekk_fit$A_sd[1,2],bekk_fit$A_sd[2,1],bekk_fit$A_sd[2,2],bekk_fit$G_sd[1,1],bekk_fit$G_sd[1,2],bekk_fit$G_sd[2,1],bekk_fit$G_sd[2,2]),
                   T_value = c(summary(var_cr)$varresult$usc$coefficients["const","t value"],summary(var_cr)$varresult$lc$coefficients["const","t value"],
                               summary(var_cr)$varresult$usc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l1","t value"],
                               summary(var_cr)$varresult$lc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l1","t value"],
                               bekk_fit$A_t[1,1],bekk_fit$A_t[1,2],bekk_fit$A_t[2,1],bekk_fit$A_t[2,2],bekk_fit$G_t[1,1],bekk_fit$G_t[1,2],
                                 bekk_fit$G_t[2,1],bekk_fit$G_t[2,2]))

bekkc$Significant_at_5p_level[abs(bekkc$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.96 & abs(bekkc$T_value) < 2.58] <- "Stat significant at 5%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.645 & abs(bekkc$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) <= 1.645] <- "Not Statistically Significant"
bekkc

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0002048468,0.0005281541,-0.3878541,Not Statistically Significant
mu20,-0.0001332705,0.0004438204,-0.3002802,Not Statistically Significant
phi11,-0.0682652464,0.0646708464,-1.0555799,Not Statistically Significant
phi12,0.0507101070,0.0770207435,0.6583954,Not Statistically Significant
phi21,-0.0370329676,0.0543444421,-0.6814490,Not Statistically Significant
phi22,0.0313572067,0.0647223528,0.4844881,Not Statistically Significant
a11,0.1091155514,0.0815565453,1.3379129,Not Statistically Significant
a12,-0.1914858789,0.0703540482,-2.7217464,Stat significant at 1%
a21,0.0797441693,0.0898028075,0.8879919,Not Statistically Significant


In [14]:
causality(var_cr,cause = "usc")$Granger
causality(var_cr,cause = "lc")$Granger


	Granger causality H0: usc do not Granger-cause lc

data:  VAR object var_cr
F-Test = 0.46437, df1 = 1, df2 = 2070, p-value = 0.4957



	Granger causality H0: lc do not Granger-cause usc

data:  VAR object var_cr
F-Test = 0.43348, df1 = 1, df2 = 2070, p-value = 0.5104


## (ii) VAR-BEKK Models for US Cocoa

In [15]:
us_d <- na.omit(merge(cr_d[,1],fr_d))
str(us_d)
head(us_d)
tail(us_d)

An xts object on 2015-12-01 / 2020-01-31 containing: 
  Data:    double [1039, 9]
  Columns: usc, eurusd, gbpusd, chfusd, usdgbp ... with 4 more columns
  Index:   Date [1039] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:3352] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "index")= num [1:3352] 1.18e+09 1.18e+09 1.18e+09 1.18e+09 1.18e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                     usc       eurusd       gbpusd       chfusd       usdgbp
2015-12-01 -0.0005991612  0.006321969  0.001625882  0.002722348 -0.001732907
2015-12-02 -0.0011994004 -0.001553197 -0.008624044  0.007359363  0.008709418
2015-12-03  0.0157764844  0.030294804  0.012891829  0.024842875 -0.012789822
2015-12-04  0.0011806377 -0.005039410 -0.002181242 -0.003532781  0.002042284
2015-12-07  0.0079330517 -0.004695709 -0.003944909 -0.003295225  0.003996838
2015-12-08 -0.0300037359  0.005338500 -0.002993915  0.008267391  0.002930901
                  eurgbp        chfgbp        ghsusd        ghsgbp
2015-12-01  4.759377e-03  0.0010835914 -0.0009211267 -0.0025456561
2015-12-02  6.216905e-03  0.0160384054  0.0012889628  0.0099212308
2015-12-03  1.717679e-02  0.0119562529 -0.0073730523 -0.0201777897
2015-12-04 -2.772965e-03 -0.0014302385  0.0113934382  0.0134894260
2015-12-07 -6.942275e-05  0.0006777364 -0.0043882219 -0.0004514615
2015-12-08  7.538854e-03  0.0111540626 -0.0096534483 -0.006

                     usc        eurusd        gbpusd        chfusd
2020-01-24 -0.0115358165 -0.0026269321 -0.0034732197 -0.0023766229
2020-01-27  0.0003625159 -0.0006351223 -0.0017219941  0.0016496850
2020-01-28 -0.0105666860  0.0003629764 -0.0020703147 -0.0034966769
2020-01-29  0.0087527911 -0.0009985023 -0.0005374487 -0.0002432794
2020-01-30  0.0272223113  0.0019053674  0.0057052210  0.0038369119
2020-01-31 -0.0103002154  0.0056042811  0.0080239246  0.0065228652
                  usdgbp        eurgbp        chfgbp        ghsusd       ghsgbp
2020-01-24  0.0035370444  0.0010679325  0.0010801539 -1.018738e-03  0.002464379
2020-01-27  0.0016985697  0.0009483168  0.0033600691  5.026346e-05  0.001733125
2020-01-28  0.0020214542  0.0024850613 -0.0013933754  2.312178e-03  0.004415901
2020-01-29  0.0005210029 -0.0005911214  0.0003168467  2.848696e-03  0.003384184
2020-01-30 -0.0056805350 -0.0035540850 -0.0019025881  9.232890e-03  0.003531378
2020-01-31 -0.0080205545 -0.0028523908 -0.001524681

## (a) US Cocoa - EURUSD

In [16]:
bekk1data <- us_d[,c(1:2)]
head(bekk1data)
tail(bekk1data)
VARselect(bekk1data,type = "const")

                     usc       eurusd
2015-12-01 -0.0005991612  0.006321969
2015-12-02 -0.0011994004 -0.001553197
2015-12-03  0.0157764844  0.030294804
2015-12-04  0.0011806377 -0.005039410
2015-12-07  0.0079330517 -0.004695709
2015-12-08 -0.0300037359  0.005338500

                     usc        eurusd
2020-01-24 -0.0115358165 -0.0026269321
2020-01-27  0.0003625159 -0.0006351223
2020-01-28 -0.0105666860  0.0003629764
2020-01-29  0.0087527911 -0.0009985023
2020-01-30  0.0272223113  0.0019053674
2020-01-31 -0.0103002154  0.0056042811

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.901054e+01,-1.900633e+01,-1.900109e+01,-1.899874e+01,-1.899454e+01,-1.898820e+01,-1.898485e+01,-1.897788e+01,-1.897649e+01,-1.897228e+01
HQ(n),-1.899962e+01,-1.898812e+01,-1.897560e+01,-1.896597e+01,-1.895449e+01,-1.894086e+01,-1.893023e+01,-1.891597e+01,-1.890730e+01,-1.889580e+01
SC(n),-1.898176e+01,-1.895836e+01,-1.893393e+01,-1.891239e+01,-1.888900e+01,-1.886348e+01,-1.884094e+01,-1.881477e+01,-1.879420e+01,-1.877079e+01
FPE(n),5.544042e-09,5.567453e-09,5.596708e-09,5.609844e-09,5.633466e-09,5.669300e-09,5.688333e-09,5.728172e-09,5.736117e-09,5.760368e-09


In [17]:
var_bekk1<-VAR(bekk1data,p = 1,type = "const")
res_bekk1 <- residuals(var_bekk1)
res_bekk1 <- as.data.frame(res_bekk1)

In [18]:
summary(var_bekk1)


VAR Estimation Results:
Endogenous variables: usc, eurusd 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6902.965 
Roots of the characteristic polynomial:
0.09648 0.006121
Call:
VAR(y = bekk1data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1    -0.0379224  0.0313982  -1.208    0.227
eurusd.l1  0.1747507  0.1178735   1.483    0.139
const     -0.0002120  0.0005277  -0.402    0.688


Residual standard error: 0.017 on 1035 degrees of freedom
Multiple R-Squared: 0.003073,	Adjusted R-squared: 0.001146 
F-statistic: 1.595 on 2 and 1035 DF,  p-value: 0.2034 


Estimation results for equation eurusd: 
eurusd = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
usc.l1     1.476e-02  8.353e-03   1.767   0.0775 .
eurusd.l1 -5.244e-02  3.136e-02  -1.672   0.0948 .
const      4.384e-05  1.404e-04   0.312   0.7549  
---
Signif. codes:  0 ‘***’

In [19]:
ind_var1 <- index(bekk1data)[-1]
str(ind_var1)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [20]:
res_bekk1 <- xts(res_bekk1,order.by = ind_var1)

In [21]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk1_fit <- bekk_fit(spec = bekk_spec,data = res_bekk1)

In [22]:
bekk1_fit$A
bekk1_fit$A_t
bekk1_fit$G
bekk1_fit$G_t

0.1486179,0.03675437
0.4707366,0.10755231


6.508308,3.381406
6.008415,3.146561


0.9191873,-0.03115894
-0.0789905,0.95797919


43.898307,-3.648638
-1.460835,44.092120


### VAR-BEKK Model Table for US Cocoa - EURUSD

In [23]:
bekk1 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk1)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Estimate"],
                                   summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Estimate"],
                                   summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Estimate"],
                                   bekk1_fit$A[1,1],bekk1_fit$A[1,2],bekk1_fit$A[2,1],bekk1_fit$A[2,2],bekk1_fit$G[1,1],bekk1_fit$G[1,2],bekk1_fit$G[2,1],bekk1_fit$G[2,2]),
                   Std_error = c(summary(var_bekk1)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Std. Error"],
                                summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Std. Error"],
                                 summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Std. Error"],
                                 bekk1_fit$A_sd[1,1],bekk1_fit$A_sd[1,2],bekk1_fit$A_sd[2,1],bekk1_fit$A_sd[2,2],bekk1_fit$G_sd[1,1],bekk1_fit$G_sd[1,2],bekk1_fit$G_sd[2,1],bekk1_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk1)$varresult$usc$coefficients["const","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["const","t value"],
                       summary(var_bekk1)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","t value"],
                               summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","t value"],
                               bekk1_fit$A_t[1,1],bekk1_fit$A_t[1,2],bekk1_fit$A_t[2,1],bekk1_fit$A_t[2,2],bekk1_fit$G_t[1,1],bekk1_fit$G_t[1,2],bekk1_fit$G_t[2,1],bekk1_fit$G_t[2,2]))

bekk1$Significant_at_5p_level[abs(bekk1$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.96 & abs(bekk1$T_value) < 2.58] <- "Stat significant at 5%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.645 & abs(bekk1$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) <= 1.645] <- "Not Statistically Significant"
bekk1

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0002119523,0.0005277312,-0.4016292,Not Statistically Significant
mu20,0.0000438417,0.0001403884,0.3122885,Not Statistically Significant
phi11,-0.0379223979,0.0313981521,-1.2077908,Not Statistically Significant
phi12,0.1747506919,0.1178735187,1.4825272,Not Statistically Significant
phi21,0.0147582855,0.0083526189,1.7669052,Stat significant at 10%
phi22,-0.0524359043,0.0313570232,-1.6722220,Stat significant at 10%
a11,0.1486179097,0.0228351082,6.5083077,Stat significant at 1%
a12,0.0367543714,0.0108695533,3.3814059,Stat significant at 1%
a21,0.4707365886,0.0783462219,6.0084147,Stat significant at 1%


In [24]:
causality(var_bekk1,cause = "usc")$Granger
causality(var_bekk1,cause = "eurusd")$Granger


	Granger causality H0: usc do not Granger-cause eurusd

data:  VAR object var_bekk1
F-Test = 3.122, df1 = 1, df2 = 2070, p-value = 0.07739



	Granger causality H0: eurusd do not Granger-cause usc

data:  VAR object var_bekk1
F-Test = 2.1979, df1 = 1, df2 = 2070, p-value = 0.1384


## (b) VAR-BEKK: US Cocoa-GBPUSD

In [25]:
bekk2data <- us_d[,c(1,3)]
head(bekk2data)
tail(bekk2data)

                     usc       gbpusd
2015-12-01 -0.0005991612  0.001625882
2015-12-02 -0.0011994004 -0.008624044
2015-12-03  0.0157764844  0.012891829
2015-12-04  0.0011806377 -0.002181242
2015-12-07  0.0079330517 -0.003944909
2015-12-08 -0.0300037359 -0.002993915

                     usc        gbpusd
2020-01-24 -0.0115358165 -0.0034732197
2020-01-27  0.0003625159 -0.0017219941
2020-01-28 -0.0105666860 -0.0020703147
2020-01-29  0.0087527911 -0.0005374487
2020-01-30  0.0272223113  0.0057052210
2020-01-31 -0.0103002154  0.0080239246

In [26]:
VARselect(bekk2data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.830155e+01,-1.829492e+01,-1.828985e+01,-1.828743e+01,-1.828368e+01,-1.828088e+01,-1.827926e+01,-1.827428e+01,-1.826682e+01,-1.825977e+01
HQ(n),-1.829062e+01,-1.827671e+01,-1.826436e+01,-1.825465e+01,-1.824362e+01,-1.823354e+01,-1.822464e+01,-1.821238e+01,-1.819764e+01,-1.818330e+01
SC(n),-1.827276e+01,-1.824694e+01,-1.822269e+01,-1.820108e+01,-1.817814e+01,-1.815615e+01,-1.813535e+01,-1.811118e+01,-1.808453e+01,-1.805829e+01
FPE(n),1.126522e-08,1.134016e-08,1.139778e-08,1.142541e-08,1.146835e-08,1.150051e-08,1.151910e-08,1.157662e-08,1.166333e-08,1.174592e-08


In [27]:
var_bekk2<-VAR(bekk2data,p = 1,type = "const")
res_bekk2 <- residuals(var_bekk2)
res_bekk2 <- as.data.frame(res_bekk2)

In [28]:
summary(var_bekk2)


VAR Estimation Results:
Endogenous variables: usc, gbpusd 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6560.008 
Roots of the characteristic polynomial:
0.04772 0.0241
Call:
VAR(y = bekk2data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + gbpusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1    -0.0404596  0.0317142  -1.276    0.202
gbpusd.l1  0.1248657  0.0850507   1.468    0.142
const     -0.0001897  0.0005278  -0.359    0.719


Residual standard error: 0.017 on 1035 degrees of freedom
Multiple R-Squared: 0.003032,	Adjusted R-squared: 0.001105 
F-statistic: 1.574 on 2 and 1035 DF,  p-value: 0.2077 


Estimation results for equation gbpusd: 
gbpusd = usc.l1 + gbpusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1     0.0037541  0.0118481   0.317    0.751
gbpusd.l1  0.0168386  0.0317739   0.530    0.596
const     -0.0001175  0.0001972  -0.596    0.551


Residual standard error: 0.006351 o

In [29]:
ind_var2 <- index(bekk2data)[-1]
str(ind_var2)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [30]:
res_bekk2 <- xts(res_bekk2,order.by = ind_var2)

In [31]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk2_fit <- bekk_fit(spec = bekk_spec,data = res_bekk2)

In [32]:
bekk2_fit$A
bekk2_fit$A_t
bekk2_fit$G
bekk2_fit$G_t

0.1268478,-0.01210763
0.1723284,0.37634284


4.289399,-0.8774239
2.211297,10.3335857


0.98654733,0.0007575778
-0.08819942,0.8790096319


110.292306,0.1022019
-2.138394,30.2388884


### VAR-BEKK Model Table for US Cocoa - GBPUSD

In [33]:
bekk2 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk2)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Estimate"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Estimate"],
                                   bekk2_fit$A[1,1],bekk2_fit$A[1,2],bekk2_fit$A[2,1],bekk2_fit$A[2,2],bekk2_fit$G[1,1],bekk2_fit$G[1,2],bekk2_fit$G[2,1],bekk2_fit$G[2,2]),
                   Std_error = c(summary(var_bekk2)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Std. Error"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Std. Error"],
                                 summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Std. Error"],
                                 bekk2_fit$A_sd[1,1],bekk2_fit$A_sd[1,2],bekk2_fit$A_sd[2,1],bekk2_fit$A_sd[2,2],bekk2_fit$G_sd[1,1],bekk2_fit$G_sd[1,2],bekk2_fit$G_sd[2,1],bekk2_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk2)$varresult$usc$coefficients["const","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","t value"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","t value"],
                               summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","t value"],
                               bekk2_fit$A_t[1,1],bekk2_fit$A_t[1,2],bekk2_fit$A_t[2,1],bekk2_fit$A_t[2,2],bekk2_fit$G_t[1,1],bekk2_fit$G_t[1,2],bekk2_fit$G_t[2,1],bekk2_fit$G_t[2,2]))

bekk2$Significant_at_5p_level[abs(bekk2$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.96 & abs(bekk2$T_value) < 2.58] <- "Stat significant at 5%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.645 & abs(bekk2$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) <= 1.645] <- "Not Statistically Significant"
bekk2

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0001897064,0.0005277987,-0.3594294,Not Statistically Significant
mu20,-0.0001174690,0.0001971793,-0.5957470,Not Statistically Significant
phi11,-0.0404596339,0.0317142307,-1.2757564,Not Statistically Significant
phi12,0.1248656825,0.0850506620,1.4681330,Not Statistically Significant
phi21,0.0037540736,0.0118480567,0.3168514,Not Statistically Significant
phi22,0.0168386034,0.0317739086,0.5299506,Not Statistically Significant
a11,0.1268477521,0.0295723852,4.2893987,Stat significant at 1%
a12,-0.0121076256,0.0137990605,-0.8774239,Not Statistically Significant
a21,0.1723284274,0.0779309388,2.2112967,Stat significant at 5%


In [34]:
causality(var_bekk2,cause = "usc")$Granger
causality(var_bekk2,cause = "gbpusd")$Granger


	Granger causality H0: usc do not Granger-cause gbpusd

data:  VAR object var_bekk2
F-Test = 0.10039, df1 = 1, df2 = 2070, p-value = 0.7514



	Granger causality H0: gbpusd do not Granger-cause usc

data:  VAR object var_bekk2
F-Test = 2.1554, df1 = 1, df2 = 2070, p-value = 0.1422


## (c) VAR-BEKK: US Cocoa-CHFUSD

In [35]:
bekk3data <- us_d[,c(1,4)]
head(bekk3data)
tail(bekk3data)

                     usc       chfusd
2015-12-01 -0.0005991612  0.002722348
2015-12-02 -0.0011994004  0.007359363
2015-12-03  0.0157764844  0.024842875
2015-12-04  0.0011806377 -0.003532781
2015-12-07  0.0079330517 -0.003295225
2015-12-08 -0.0300037359  0.008267391

                     usc        chfusd
2020-01-24 -0.0115358165 -0.0023766229
2020-01-27  0.0003625159  0.0016496850
2020-01-28 -0.0105666860 -0.0034966769
2020-01-29  0.0087527911 -0.0002432794
2020-01-30  0.0272223113  0.0038369119
2020-01-31 -0.0103002154  0.0065228652

In [36]:
VARselect(bekk3data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.905417e+01,-1.905170e+01,-1.905318e+01,-1.905156e+01,-1.904814e+01,-1.904307e+01,-1.904058e+01,-1.903825e+01,-1.903585e+01,-1.903082e+01
HQ(n),-1.904324e+01,-1.903349e+01,-1.902769e+01,-1.901879e+01,-1.900809e+01,-1.899573e+01,-1.898596e+01,-1.897634e+01,-1.896666e+01,-1.895435e+01
SC(n),-1.902538e+01,-1.900373e+01,-1.898602e+01,-1.896521e+01,-1.894261e+01,-1.891834e+01,-1.889667e+01,-1.887514e+01,-1.885356e+01,-1.882934e+01
FPE(n),5.307391e-09,5.320499e-09,5.312631e-09,5.321233e-09,5.339451e-09,5.366624e-09,5.379996e-09,5.392592e-09,5.405537e-09,5.432795e-09


In [37]:
var_bekk3<-VAR(bekk3data,p = 1,type = "const")
res_bekk3 <- residuals(var_bekk3)
res_bekk3 <- as.data.frame(res_bekk3)


In [38]:
ind_var3 <- index(bekk3data)[-1]
str(ind_var3)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [39]:
res_bekk3 <- xts(res_bekk3,order.by = ind_var3)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk3_fit <- bekk_fit(spec = bekk_spec,data = res_bekk3)

In [40]:
bekk3_fit$A
bekk3_fit$A_t
bekk3_fit$G
bekk3_fit$G_t

0.1810719,0.02832126
0.4812738,0.18224912


4.612696,1.78682
3.114373,2.89646


0.9361388,-0.02373348
-0.3411660,0.82974313


30.290464,-1.732457
-1.754248,18.232282


### VAR-BEKK Model Table for US Cocoa - CHFUSD

In [41]:
bekk3 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk3)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Estimate"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Estimate"],
                                   summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Estimate"],
                                   bekk3_fit$A[1,1],bekk3_fit$A[1,2],bekk3_fit$A[2,1],bekk3_fit$A[2,2],bekk3_fit$G[1,1],bekk3_fit$G[1,2],bekk3_fit$G[2,1],bekk3_fit$G[2,2]),
                   Std_error = c(summary(var_bekk3)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Std. Error"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Std. Error"],
                                 summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Std. Error"],
                                 bekk3_fit$A_sd[1,1],bekk3_fit$A_sd[1,2],bekk3_fit$A_sd[2,1],bekk3_fit$A_sd[2,2],bekk3_fit$G_sd[1,1],bekk3_fit$G_sd[1,2],bekk3_fit$G_sd[2,1],bekk3_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk3)$varresult$usc$coefficients["const","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["const","t value"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","t value"],
                               summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","t value"],
                               bekk3_fit$A_t[1,1],bekk3_fit$A_t[1,2],bekk3_fit$A_t[2,1],bekk3_fit$A_t[2,2],bekk3_fit$G_t[1,1],bekk3_fit$G_t[1,2],bekk3_fit$G_t[2,1],bekk3_fit$G_t[2,2]))

bekk3$Significant_at_5p_level[abs(bekk3$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.96 & abs(bekk3$T_value) < 2.58] <- "Stat significant at 5%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.645 & abs(bekk3$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) <= 1.645] <- "Not Statistically Significant"
bekk3

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-2.233498e-04,0.0005272743,-0.42359324,Not Statistically Significant
mu20,7.888269e-05,0.0001354403,0.58241678,Not Statistically Significant
phi11,-3.620364e-02,0.0311170986,-1.16346460,Not Statistically Significant
phi12,2.496969e-01,0.1215167223,2.05483589,Stat significant at 5%
phi21,3.805156e-03,0.0079930086,0.47606056,Not Statistically Significant
phi22,2.715651e-03,0.0312138423,0.08700148,Not Statistically Significant
a11,1.810719e-01,0.0392551053,4.61269563,Stat significant at 1%
a12,2.832126e-02,0.0158500883,1.78682027,Stat significant at 10%
a21,4.812738e-01,0.1545331352,3.11437307,Stat significant at 1%


In [42]:
causality(var_bekk3,cause = "usc")$Granger
causality(var_bekk3,cause = "chfusd")$Granger


	Granger causality H0: usc do not Granger-cause chfusd

data:  VAR object var_bekk3
F-Test = 0.22663, df1 = 1, df2 = 2070, p-value = 0.6341



	Granger causality H0: chfusd do not Granger-cause usc

data:  VAR object var_bekk3
F-Test = 4.2224, df1 = 1, df2 = 2070, p-value = 0.04002


## (d) VAR-BEKK model for US Cocoa - GHSUSD

In [43]:
bekk7data <- na.omit(merge(cr_d[,1],fr_d[,7]))
head(bekk7data)
tail(bekk7data)

                     usc        ghsusd
2015-12-01 -0.0005991612 -0.0009211267
2015-12-02 -0.0011994004  0.0012889628
2015-12-03  0.0157764844 -0.0073730523
2015-12-04  0.0011806377  0.0113934382
2015-12-07  0.0079330517 -0.0043882219
2015-12-08 -0.0300037359 -0.0096534483

                     usc        ghsusd
2020-01-24 -0.0115358165 -1.018738e-03
2020-01-27  0.0003625159  5.026346e-05
2020-01-28 -0.0105666860  2.312178e-03
2020-01-29  0.0087527911  2.848696e-03
2020-01-30  0.0272223113  9.232890e-03
2020-01-31 -0.0103002154  3.445617e-03

In [44]:
VARselect(bekk7data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.765576e+01,-1.765187e+01,-1.764769e+01,-1.764199e+01,-1.763881e+01,-1.763422e+01,-1.763252e+01,-1.762693e+01,-1.762290e+01,-1.761814e+01
HQ(n),-1.764484e+01,-1.763366e+01,-1.762220e+01,-1.760922e+01,-1.759875e+01,-1.758688e+01,-1.757790e+01,-1.756502e+01,-1.755371e+01,-1.754167e+01
SC(n),-1.762698e+01,-1.760390e+01,-1.758052e+01,-1.755564e+01,-1.753327e+01,-1.750949e+01,-1.748861e+01,-1.746382e+01,-1.744061e+01,-1.741666e+01
FPE(n),2.148814e-08,2.157195e-08,2.166245e-08,2.178622e-08,2.185560e-08,2.195621e-08,2.199350e-08,2.211692e-08,2.220620e-08,2.231225e-08


In [45]:
var_bekk7<-VAR(bekk7data,p = 1,type = "const")
res_bekk7 <- residuals(var_bekk7)
res_bekk7 <- as.data.frame(res_bekk7)

In [46]:
summary(var_bekk7)


VAR Estimation Results:
Endogenous variables: usc, ghsusd 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6225.706 
Roots of the characteristic polynomial:
0.3835 0.02973
Call:
VAR(y = bekk7data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + ghsusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1    -0.0311082  0.0310785  -1.001    0.317
ghsusd.l1  0.0174659  0.0569626   0.307    0.759
const     -0.0001988  0.0005285  -0.376    0.707


Residual standard error: 0.01702 on 1035 degrees of freedom
Multiple R-Squared: 0.001047,	Adjusted R-squared: -0.0008838 
F-statistic: 0.5422 on 2 and 1035 DF,  p-value: 0.5817 


Estimation results for equation ghsusd: 
ghsusd = usc.l1 + ghsusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)    
usc.l1     0.0278664  0.0156597   1.779   0.0755 .  
ghsusd.l1 -0.3820885  0.0287021 -13.312   <2e-16 ***
const     -0.0003793  0.0002663  -1.425   0.1546    
---
Signif. code

In [47]:
ind_var7 <- index(bekk7data)[-1]
str(ind_var7)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [48]:
res_bekk7 <- xts(res_bekk7,order.by = ind_var7)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk7_fit <- bekk_fit(spec = bekk_spec,data = res_bekk7)

In [49]:
bekk7_fit$A
bekk7_fit$A_t
bekk7_fit$G
bekk7_fit$G_t

0.11089403,-0.01779038
-0.03388237,0.40015733


4.5559544,-1.796756
-0.8000931,17.961708


0.989050204,0.000681222
0.007781085,0.905998115


145.5949703,0.08261115
0.4056989,107.05134849


### VAR-BEKK Model Table for US Cocoa - GHSUSD

In [50]:
bekk7 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk7)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Estimate"],
                                   bekk7_fit$A[1,1],bekk7_fit$A[1,2],bekk7_fit$A[2,1],bekk7_fit$A[2,2],bekk7_fit$G[1,1],bekk7_fit$G[1,2],bekk7_fit$G[2,1],bekk7_fit$G[2,2]),
                   Std_error = c(summary(var_bekk7)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Std. Error"],
                                   bekk7_fit$A_sd[1,1],bekk7_fit$A_sd[1,2],bekk7_fit$A_sd[2,1],bekk7_fit$A_sd[2,2],bekk7_fit$G_sd[1,1],bekk7_fit$G_sd[1,2],bekk7_fit$G_sd[2,1],bekk7_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk7)$varresult$usc$coefficients["const","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","t value"],
                                   bekk7_fit$A_t[1,1],bekk7_fit$A_t[1,2],bekk7_fit$A_t[2,1],bekk7_fit$A_t[2,2],bekk7_fit$G_t[1,1],bekk7_fit$G_t[1,2],bekk7_fit$G_t[2,1],bekk7_fit$G_t[2,2]))

bekk7$Significant_at_5p_level[abs(bekk7$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.96 & abs(bekk7$T_value) < 2.58] <- "Stat significant at 5%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.645 & abs(bekk7$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) <= 1.645] <- "Not Statistically Significant"
bekk7

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0001988219,0.0005284774,-0.37621647,Not Statistically Significant
mu20,-0.0003793294,0.0002662874,-1.42451125,Not Statistically Significant
phi11,-0.0311081651,0.0310784881,-1.00095491,Not Statistically Significant
phi12,0.0174659052,0.0569625699,0.30662074,Not Statistically Significant
phi21,0.0278664387,0.0156597247,1.77949736,Stat significant at 10%
phi22,-0.3820885366,0.0287021093,-13.31221107,Stat significant at 1%
a11,0.1108940332,0.0243404618,4.55595436,Stat significant at 1%
a12,-0.0177903815,0.0099013873,-1.79675644,Stat significant at 10%
a21,-0.0338823681,0.0423480335,-0.80009307,Not Statistically Significant


In [51]:
causality(var_bekk7,cause = "usc")$Granger
causality(var_bekk7,cause = "ghsusd")$Granger


	Granger causality H0: usc do not Granger-cause ghsusd

data:  VAR object var_bekk7
F-Test = 3.1666, df1 = 1, df2 = 2070, p-value = 0.0753



	Granger causality H0: ghsusd do not Granger-cause usc

data:  VAR object var_bekk7
F-Test = 0.094016, df1 = 1, df2 = 2070, p-value = 0.7592


## (iii) VAR-BEKK Models for London Cocoa

In [52]:
l_d <- na.omit(merge(cr_d[,2],fr_d[,c(4:6)]))
str(l_d)
head(l_d)
tail(l_d)

An xts object on 2015-12-01 / 2020-01-31 containing: 
  Data:    double [1039, 4]
  Columns: lc, usdgbp, eurgbp, chfgbp
  Index:   Date [1039] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:3352] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "index")= num [1:3352] 1.18e+09 1.18e+09 1.18e+09 1.18e+09 1.18e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                     lc       usdgbp        eurgbp        chfgbp
2015-12-01 -0.000877193 -0.001732907  4.759377e-03  0.0010835914
2015-12-02  0.007431728  0.008709418  6.216905e-03  0.0160384054
2015-12-03  0.005646052 -0.012789822  1.717679e-02  0.0119562529
2015-12-04  0.001730853  0.002042284 -2.772965e-03 -0.0014302385
2015-12-07  0.005604671  0.003996838 -6.942275e-05  0.0006777364
2015-12-08 -0.019536111  0.002930901  7.538854e-03  0.0111540626

                     lc        usdgbp        eurgbp        chfgbp
2020-01-24 -0.004590674  0.0035370444  0.0010679325  0.0010801539
2020-01-27  0.001532567  0.0016985697  0.0009483168  0.0033600691
2020-01-28 -0.004092077  0.0020214542  0.0024850613 -0.0013933754
2020-01-29  0.005112486  0.0005210029 -0.0005911214  0.0003168467
2020-01-30  0.019692633 -0.0056805350 -0.0035540850 -0.0019025881
2020-01-31 -0.015113638 -0.0080205545 -0.0028523908 -0.0015246811

## (a) VAR-BEKK: London-USDGBP

In [53]:
bekk4data <- l_d[,c(1,2)]
head(bekk4data)
tail(bekk4data)

                     lc       usdgbp
2015-12-01 -0.000877193 -0.001732907
2015-12-02  0.007431728  0.008709418
2015-12-03  0.005646052 -0.012789822
2015-12-04  0.001730853  0.002042284
2015-12-07  0.005604671  0.003996838
2015-12-08 -0.019536111  0.002930901

                     lc        usdgbp
2020-01-24 -0.004590674  0.0035370444
2020-01-27  0.001532567  0.0016985697
2020-01-28 -0.004092077  0.0020214542
2020-01-29  0.005112486  0.0005210029
2020-01-30  0.019692633 -0.0056805350
2020-01-31 -0.015113638 -0.0080205545

In [54]:
VARselect(bekk4data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.861441e+01,-1.860735e+01,-1.860285e+01,-1.859968e+01,-1.859523e+01,-1.859297e+01,-1.858787e+01,-1.858210e+01,-1.857466e+01,-1.856819e+01
HQ(n),-1.860349e+01,-1.858914e+01,-1.857736e+01,-1.856690e+01,-1.855517e+01,-1.854563e+01,-1.853324e+01,-1.852019e+01,-1.850547e+01,-1.849172e+01
SC(n),-1.858563e+01,-1.855937e+01,-1.853569e+01,-1.851333e+01,-1.848969e+01,-1.846825e+01,-1.844395e+01,-1.841899e+01,-1.839237e+01,-1.836671e+01
FPE(n),8.238800e-09,8.297212e-09,8.334634e-09,8.361097e-09,8.398361e-09,8.417353e-09,8.460453e-09,8.509403e-09,8.572959e-09,8.628647e-09


In [55]:
var_bekk4<-VAR(bekk4data,p = 1,type = "const")
res_bekk4 <- residuals(var_bekk4)
res_bekk4 <- as.data.frame(res_bekk4)

In [56]:
summary(var_bekk4)


VAR Estimation Results:
Endogenous variables: lc, usdgbp 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6722.881 
Roots of the characteristic polynomial:
0.01114 0.003661
Call:
VAR(y = bekk4data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + usdgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0107516  0.0312103  -0.344    0.731
usdgbp.l1  0.0838294  0.0702998   1.192    0.233
const     -0.0001417  0.0004437  -0.319    0.749


Residual standard error: 0.01429 on 1035 degrees of freedom
Multiple R-Squared: 0.001426,	Adjusted R-squared: -0.0005041 
F-statistic: 0.7388 on 2 and 1035 DF,  p-value: 0.478 


Estimation results for equation usdgbp: 
usdgbp = lc.l1 + usdgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0018515  0.0138668  -0.134    0.894
usdgbp.l1  0.0182278  0.0312344   0.584    0.560
const      0.0001171  0.0001971   0.594    0.553


Residual standard error: 0.00635 o

In [57]:
ind_var4 <- index(bekk4data)[-1]
str(ind_var4)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [58]:
res_bekk4 <- xts(res_bekk4,order.by = ind_var4)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk4_fit <- bekk_fit(spec = bekk_spec,data = res_bekk4)

In [59]:
bekk4_fit$A
bekk4_fit$A_t
bekk4_fit$G
bekk4_fit$G_t

0.17012317,0.0001041963
0.02507592,0.3669277581


5.5851502,0.006110939
0.3068789,12.854606608


0.974258170,0.002516031
0.007762627,0.875523812


89.9161661,0.2967775
0.2127717,34.0888395


### VAR-BEKK Model Table for London Cocoa - USDGBP

In [60]:
bekk4 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk4)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Estimate"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Estimate"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Estimate"],
                                   bekk4_fit$A[1,1],bekk4_fit$A[1,2],bekk4_fit$A[2,1],bekk4_fit$A[2,2],bekk4_fit$G[1,1],bekk4_fit$G[1,2],bekk4_fit$G[2,1],bekk4_fit$G[2,2]),
                   Std_error = c(summary(var_bekk4)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Std. Error"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Std. Error"],
                                   bekk4_fit$A_sd[1,1],bekk4_fit$A_sd[1,2],bekk4_fit$A_sd[2,1],bekk4_fit$A_sd[2,2],bekk4_fit$G_sd[1,1],bekk4_fit$G_sd[1,2],bekk4_fit$G_sd[2,1],bekk4_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk4)$varresult$lc$coefficients["const","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","t value"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","t value"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","t value"],
                                   bekk4_fit$A_t[1,1],bekk4_fit$A_t[1,2],bekk4_fit$A_t[2,1],bekk4_fit$A_t[2,2],bekk4_fit$G_t[1,1],bekk4_fit$G_t[1,2],bekk4_fit$G_t[2,1],bekk4_fit$G_t[2,2]))

bekk4$Significant_at_5p_level[abs(bekk4$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.96 & abs(bekk4$T_value) < 2.58] <- "Stat significant at 5%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.645 & abs(bekk4$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) <= 1.645] <- "Not Statistically Significant"
bekk4

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0001417143,0.0004436950,-0.319395732,Not Statistically Significant
mu20,0.0001170693,0.0001971347,0.593854350,Not Statistically Significant
phi11,-0.0107516470,0.0312103423,-0.344489876,Not Statistically Significant
phi12,0.0838293585,0.0702998295,1.192454648,Not Statistically Significant
phi21,-0.0018514870,0.0138668241,-0.133519180,Not Statistically Significant
phi22,0.0182277749,0.0312343697,0.583580686,Not Statistically Significant
a11,0.1701231705,0.0304599095,5.585150229,Stat significant at 1%
a12,0.0001041963,0.0170507901,0.006110939,Not Statistically Significant
a21,0.0250759178,0.0817127575,0.306878858,Not Statistically Significant


In [61]:
causality(var_bekk4,cause = "lc")$Granger
causality(var_bekk4,cause = "usdgbp")$Granger


	Granger causality H0: lc do not Granger-cause usdgbp

data:  VAR object var_bekk4
F-Test = 0.017827, df1 = 1, df2 = 2070, p-value = 0.8938



	Granger causality H0: usdgbp do not Granger-cause lc

data:  VAR object var_bekk4
F-Test = 1.4219, df1 = 1, df2 = 2070, p-value = 0.2332


## (b) VAR-BEKK: London Cocoa-EURGBP

In [62]:
bekk5data <- l_d[,c(1,3)]
head(bekk5data)

                     lc        eurgbp
2015-12-01 -0.000877193  4.759377e-03
2015-12-02  0.007431728  6.216905e-03
2015-12-03  0.005646052  1.717679e-02
2015-12-04  0.001730853 -2.772965e-03
2015-12-07  0.005604671 -6.942275e-05
2015-12-08 -0.019536111  7.538854e-03

In [63]:
VARselect(bekk5data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.893668e+01,-1.892916e+01,-1.892392e+01,-1.891716e+01,-1.891106e+01,-1.890844e+01,-1.890494e+01,-1.890064e+01,-1.889347e+01,-1.888787e+01
HQ(n),-1.892576e+01,-1.891095e+01,-1.889843e+01,-1.888438e+01,-1.887101e+01,-1.886110e+01,-1.885032e+01,-1.883873e+01,-1.882429e+01,-1.881140e+01
SC(n),-1.890790e+01,-1.888118e+01,-1.885676e+01,-1.883081e+01,-1.880553e+01,-1.878371e+01,-1.876103e+01,-1.873753e+01,-1.871118e+01,-1.868638e+01
FPE(n),5.969009e-09,6.014113e-09,6.045672e-09,6.086719e-09,6.123922e-09,6.140044e-09,6.161534e-09,6.188147e-09,6.232641e-09,6.267707e-09


In [64]:
var_bekk5<-VAR(bekk5data,p = 1,type = "const")
res_bekk5 <- residuals(var_bekk5)
res_bekk5 <- as.data.frame(res_bekk5)

In [65]:
summary(var_bekk5)


VAR Estimation Results:
Endogenous variables: lc, eurgbp 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6888.327 
Roots of the characteristic polynomial:
0.05144 0.03136
Call:
VAR(y = bekk5data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + eurgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0116334  0.0312781  -0.372    0.710
eurgbp.l1  0.1001052  0.0822888   1.217    0.224
const     -0.0001491  0.0004438  -0.336    0.737


Residual standard error: 0.01429 on 1035 degrees of freedom
Multiple R-Squared: 0.001481,	Adjusted R-squared: -0.0004482 
F-statistic: 0.7677 on 2 and 1035 DF,  p-value: 0.4643 


Estimation results for equation eurgbp: 
eurgbp = lc.l1 + eurgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)
lc.l1     0.0124271  0.0118753   1.046    0.296
eurgbp.l1 0.0317106  0.0312424   1.015    0.310
const     0.0001661  0.0001685   0.986    0.324


Residual standard error: 0.005426 on 1

In [66]:
ind_var5 <- index(bekk5data)[-1]
str(ind_var5)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [67]:
res_bekk5 <- xts(res_bekk5,order.by = ind_var5)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk5_fit <- bekk_fit(spec = bekk_spec,data = res_bekk5)

In [68]:
bekk5_fit$A
bekk5_fit$A_t
bekk5_fit$G
bekk5_fit$G_t

0.1718022,0.005009541
0.0403958,0.288542333


5.312287,0.3455693
0.446414,10.5612721


0.97190120,0.001462234
0.00514429,0.937896418


76.7648522,0.2164404
0.1542304,70.7669322


### VAR-BEKK Model Table for London-EURGBP

In [69]:
bekk5 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk5)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Estimate"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Estimate"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Estimate"],
                                   bekk5_fit$A[1,1],bekk5_fit$A[1,2],bekk5_fit$A[2,1],bekk5_fit$A[2,2],bekk5_fit$G[1,1],bekk5_fit$G[1,2],bekk5_fit$G[2,1],bekk5_fit$G[2,2]),
                   Std_error = c(summary(var_bekk5)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Std. Error"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Std. Error"],
                                   bekk5_fit$A_sd[1,1],bekk5_fit$A_sd[1,2],bekk5_fit$A_sd[2,1],bekk5_fit$A_sd[2,2],bekk5_fit$G_sd[1,1],bekk5_fit$G_sd[1,2],bekk5_fit$G_sd[2,1],bekk5_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk5)$varresult$lc$coefficients["const","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","t value"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","t value"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","t value"],
                                   bekk5_fit$A_t[1,1],bekk5_fit$A_t[1,2],bekk5_fit$A_t[2,1],bekk5_fit$A_t[2,2],bekk5_fit$G_t[1,1],bekk5_fit$G_t[1,2],bekk5_fit$G_t[2,1],bekk5_fit$G_t[2,2]))

bekk5$Significant_at_5p_level[abs(bekk5$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.96 & abs(bekk5$T_value) < 2.58] <- "Stat significant at 5%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.645 & abs(bekk5$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) <= 1.645] <- "Not Statistically Significant"
bekk5

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0001490710,0.0004438426,-0.3358646,Not Statistically Significant
mu20,0.0001661489,0.0001685130,0.9859708,Not Statistically Significant
phi11,-0.0116334050,0.0312780532,-0.3719351,Not Statistically Significant
phi12,0.1001051931,0.0822887670,1.2165110,Not Statistically Significant
phi21,0.0124271472,0.0118752882,1.0464712,Not Statistically Significant
phi22,0.0317106202,0.0312424440,1.0149853,Not Statistically Significant
a11,0.1718021798,0.0323405318,5.3122868,Stat significant at 1%
a12,0.0050095413,0.0144964899,0.3455693,Not Statistically Significant
a21,0.0403958038,0.0904895449,0.4464140,Not Statistically Significant


In [70]:
causality(var_bekk5,cause = "lc")$Granger
causality(var_bekk5,cause = "eurgbp")$Granger


	Granger causality H0: lc do not Granger-cause eurgbp

data:  VAR object var_bekk5
F-Test = 1.0951, df1 = 1, df2 = 2070, p-value = 0.2955



	Granger causality H0: eurgbp do not Granger-cause lc

data:  VAR object var_bekk5
F-Test = 1.4799, df1 = 1, df2 = 2070, p-value = 0.2239


## (c) VAR-BEKK: London-CHFGBP

In [71]:
bekk6data <- l_d[,c(1,4)]
head(bekk6data)
tail(bekk6data)

                     lc        chfgbp
2015-12-01 -0.000877193  0.0010835914
2015-12-02  0.007431728  0.0160384054
2015-12-03  0.005646052  0.0119562529
2015-12-04  0.001730853 -0.0014302385
2015-12-07  0.005604671  0.0006777364
2015-12-08 -0.019536111  0.0111540626

                     lc        chfgbp
2020-01-24 -0.004590674  0.0010801539
2020-01-27  0.001532567  0.0033600691
2020-01-28 -0.004092077 -0.0013933754
2020-01-29  0.005112486  0.0003168467
2020-01-30  0.019692633 -0.0019025881
2020-01-31 -0.015113638 -0.0015246811

In [72]:
VARselect(bekk6data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.872323e+01,-1.871636e+01,-1.871067e+01,-1.870472e+01,-1.869931e+01,-1.869726e+01,-1.869152e+01,-1.868484e+01,-1.867717e+01,-1.867240e+01
HQ(n),-1.871231e+01,-1.869815e+01,-1.868518e+01,-1.867195e+01,-1.865925e+01,-1.864992e+01,-1.863690e+01,-1.862294e+01,-1.860798e+01,-1.859593e+01
SC(n),-1.869445e+01,-1.866839e+01,-1.864351e+01,-1.861837e+01,-1.859377e+01,-1.857254e+01,-1.854760e+01,-1.852174e+01,-1.849488e+01,-1.847091e+01
FPE(n),7.389316e-09,7.440261e-09,7.482694e-09,7.527360e-09,7.568231e-09,7.583725e-09,7.627416e-09,7.678524e-09,7.737695e-09,7.774741e-09


In [73]:
var_bekk6<-VAR(bekk6data,p = 1,type = "const")
res_bekk6 <- residuals(var_bekk6)
res_bekk6 <- as.data.frame(res_bekk6)

In [74]:
summary(var_bekk6)


VAR Estimation Results:
Endogenous variables: lc, chfgbp 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6777.723 
Roots of the characteristic polynomial:
0.0317 0.01445
Call:
VAR(y = bekk6data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + chfgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0106410  0.0311866  -0.341    0.733
chfgbp.l1  0.0932514  0.0740389   1.259    0.208
const     -0.0001498  0.0004438  -0.338    0.736


Residual standard error: 0.01429 on 1035 degrees of freedom
Multiple R-Squared: 0.001584,	Adjusted R-squared: -0.0003455 
F-statistic: 0.8209 on 2 and 1035 DF,  p-value: 0.4403 


Estimation results for equation chfgbp: 
chfgbp = lc.l1 + chfgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)
lc.l1     0.0017298  0.0131349   0.132    0.895
chfgbp.l1 0.0278947  0.0311830   0.895    0.371
const     0.0001922  0.0001869   1.028    0.304


Residual standard error: 0.006019 on 10

In [75]:
ind_var6 <- index(bekk6data)[-1]
str(ind_var6)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [76]:
res_bekk6 <- xts(res_bekk6,order.by = ind_var6)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk6_fit <- bekk_fit(spec = bekk_spec,data = res_bekk6)

In [77]:
bekk6_fit$A
bekk6_fit$A_t
bekk6_fit$G
bekk6_fit$G_t

0.16408630,0.02526927
0.03049935,0.37759282


5.9807949,1.632192
0.4008725,12.287750


0.976013296,-0.006099318
0.006261844,0.887154300


100.7548576,-0.7089993
0.2088587,41.2113063


### VAR-BEKK Model Table for London-CHFGBP

In [78]:
bekk6 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk6)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Estimate"],
                                   bekk6_fit$A[1,1],bekk6_fit$A[1,2],bekk6_fit$A[2,1],bekk6_fit$A[2,2],bekk6_fit$G[1,1],bekk6_fit$G[1,2],bekk6_fit$G[2,1],bekk6_fit$G[2,2]),
                   Std_error = c(summary(var_bekk6)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Std. Error"],
                                   bekk6_fit$A_sd[1,1],bekk6_fit$A_sd[1,2],bekk6_fit$A_sd[2,1],bekk6_fit$A_sd[2,2],bekk6_fit$G_sd[1,1],bekk6_fit$G_sd[1,2],bekk6_fit$G_sd[2,1],bekk6_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk6)$varresult$lc$coefficients["const","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","t value"],
                                   bekk6_fit$A_t[1,1],bekk6_fit$A_t[1,2],bekk6_fit$A_t[2,1],bekk6_fit$A_t[2,2],bekk6_fit$G_t[1,1],bekk6_fit$G_t[1,2],bekk6_fit$G_t[2,1],bekk6_fit$G_t[2,2]))

bekk6$Significant_at_5p_level[abs(bekk6$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.96 & abs(bekk6$T_value) < 2.58] <- "Stat significant at 5%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.645 & abs(bekk6$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) <= 1.645] <- "Not Statistically Significant"
bekk6

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0001498321,0.0004438229,-0.3375942,Not Statistically Significant
mu20,0.0001922343,0.0001869253,1.0284017,Not Statistically Significant
phi11,-0.0106409769,0.0311866387,-0.3412031,Not Statistically Significant
phi12,0.0932513518,0.0740389300,1.2594908,Not Statistically Significant
phi21,0.0017297882,0.0131349050,0.1316940,Not Statistically Significant
phi22,0.0278946885,0.0311830435,0.8945467,Not Statistically Significant
a11,0.1640862981,0.0274355332,5.9807949,Stat significant at 1%
a12,0.0252692708,0.0154817965,1.6321924,Not Statistically Significant
a21,0.0304993505,0.0760824265,0.4008725,Not Statistically Significant


In [79]:
causality(var_bekk6,cause = "lc")$Granger
causality(var_bekk6,cause = "chfgbp")$Granger


	Granger causality H0: lc do not Granger-cause chfgbp

data:  VAR object var_bekk6
F-Test = 0.017343, df1 = 1, df2 = 2070, p-value = 0.8952



	Granger causality H0: chfgbp do not Granger-cause lc

data:  VAR object var_bekk6
F-Test = 1.5863, df1 = 1, df2 = 2070, p-value = 0.208


## (d) VAR-BEKK Model for London Cocoa - GHSGBP

In [80]:
bekk8data <- na.omit(merge(cr_d[,2],fr_d[,8]))
head(bekk8data)
tail(bekk8data)

                     lc        ghsgbp
2015-12-01 -0.000877193 -0.0025456561
2015-12-02  0.007431728  0.0099212308
2015-12-03  0.005646052 -0.0201777897
2015-12-04  0.001730853  0.0134894260
2015-12-07  0.005604671 -0.0004514615
2015-12-08 -0.019536111 -0.0066621131

                     lc       ghsgbp
2020-01-24 -0.004590674  0.002464379
2020-01-27  0.001532567  0.001733125
2020-01-28 -0.004092077  0.004415901
2020-01-29  0.005112486  0.003384184
2020-01-30  0.019692633  0.003531378
2020-01-31 -0.015113638 -0.004588158

In [81]:
VARselect(bekk8data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.752230e+01,-1.751593e+01,-1.751058e+01,-1.751137e+01,-1.750772e+01,-1.750222e+01,-1.749956e+01,-1.749306e+01,-1.748761e+01,-1.748142e+01
HQ(n),-1.751138e+01,-1.749772e+01,-1.748509e+01,-1.747860e+01,-1.746766e+01,-1.745488e+01,-1.744494e+01,-1.743116e+01,-1.741842e+01,-1.740495e+01
SC(n),-1.749352e+01,-1.746795e+01,-1.744342e+01,-1.742502e+01,-1.740218e+01,-1.737749e+01,-1.735565e+01,-1.732996e+01,-1.730532e+01,-1.727994e+01
FPE(n),2.455620e-08,2.471328e-08,2.484580e-08,2.482608e-08,2.491698e-08,2.505436e-08,2.512108e-08,2.528490e-08,2.542323e-08,2.558118e-08


In [82]:
var_bekk8<-VAR(bekk8data,p = 1,type = "const")
res_bekk8 <- residuals(var_bekk8)
res_bekk8 <- as.data.frame(res_bekk8)

In [83]:
summary(var_bekk8)


VAR Estimation Results:
Endogenous variables: lc, ghsgbp 
Deterministic variables: const 
Sample size: 1038 
Log Likelihood: 6158.023 
Roots of the characteristic polynomial:
0.2756 0.005368
Call:
VAR(y = bekk8data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + ghsgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0087915  0.0311440  -0.282    0.778
ghsgbp.l1  0.0312662  0.0392391   0.797    0.426
const     -0.0001261  0.0004438  -0.284    0.776


Residual standard error: 0.0143 on 1035 degrees of freedom
Multiple R-Squared: 0.0006666,	Adjusted R-squared: -0.001264 
F-statistic: 0.3452 on 2 and 1035 DF,  p-value: 0.7082 


Estimation results for equation ghsgbp: 
ghsgbp = lc.l1 + ghsgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)    
lc.l1      0.0292071  0.0237805   1.228    0.220    
ghsgbp.l1 -0.2721377  0.0299616  -9.083   <2e-16 ***
const     -0.0001965  0.0003389  -0.580    0.562    
---
Signif. codes:  0

In [84]:
ind_var8 <- index(bekk8data)[-1]
str(ind_var8)

 Date[1:1038], format: "2015-12-02" "2015-12-03" "2015-12-04" "2015-12-07" "2015-12-08" ...


In [85]:
res_bekk8 <- xts(res_bekk8,order.by = ind_var8)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk8_fit <- bekk_fit(spec = bekk_spec,data = res_bekk8)

In [86]:
bekk8_fit$A
bekk8_fit$A_t
bekk8_fit$G
bekk8_fit$G_t

0.16181746,0.08088821
-0.01618575,0.39774999


5.3479359,3.988333
-0.4587112,16.207945


0.974735198,-0.01257916
-0.003215805,0.89499767


89.3073457,-0.8833952
-0.1777962,68.7900815


### VAR-BEKK Model Table for London Cocoa - GHSGBP

In [87]:
bekk8 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk8)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Estimate"],
                                   bekk8_fit$A[1,1],bekk8_fit$A[1,2],bekk8_fit$A[2,1],bekk8_fit$A[2,2],bekk8_fit$G[1,1],bekk8_fit$G[1,2],bekk8_fit$G[2,1],bekk8_fit$G[2,2]),
                   Std_error = c(summary(var_bekk8)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Std. Error"],
                                   bekk8_fit$A_sd[1,1],bekk8_fit$A_sd[1,2],bekk8_fit$A_sd[2,1],bekk8_fit$A_sd[2,2],bekk8_fit$G_sd[1,1],bekk8_fit$G_sd[1,2],bekk8_fit$G_sd[2,1],bekk8_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk8)$varresult$lc$coefficients["const","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","t value"],
                                   bekk8_fit$A_t[1,1],bekk8_fit$A_t[1,2],bekk8_fit$A_t[2,1],bekk8_fit$A_t[2,2],bekk8_fit$G_t[1,1],bekk8_fit$G_t[1,2],bekk8_fit$G_t[2,1],bekk8_fit$G_t[2,2]))

bekk8$Significant_at_5p_level[abs(bekk8$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.96 & abs(bekk8$T_value) < 2.58] <- "Stat significant at 5%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.645 & abs(bekk8$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) <= 1.645] <- "Not Statistically Significant"
bekk8

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,-0.0001260913,0.0004438080,-0.2841122,Not Statistically Significant
mu20,-0.0001964596,0.0003388764,-0.5797383,Not Statistically Significant
phi11,-0.0087914776,0.0311440053,-0.2822847,Not Statistically Significant
phi12,0.0312662142,0.0392391144,0.7968124,Not Statistically Significant
phi21,0.0292070711,0.0237804815,1.2281951,Not Statistically Significant
phi22,-0.2721377420,0.0299616259,-9.0828763,Stat significant at 1%
a11,0.1618174569,0.0302579273,5.3479359,Stat significant at 1%
a12,0.0808882097,0.0202812065,3.9883332,Stat significant at 1%
a21,-0.0161857539,0.0352852849,-0.4587112,Not Statistically Significant


In [88]:
causality(var_bekk8,cause = "lc")$Granger
causality(var_bekk8,cause = "ghsgbp")$Granger


	Granger causality H0: lc do not Granger-cause ghsgbp

data:  VAR object var_bekk8
F-Test = 1.5085, df1 = 1, df2 = 2070, p-value = 0.2195



	Granger causality H0: ghsgbp do not Granger-cause lc

data:  VAR object var_bekk8
F-Test = 0.63491, df1 = 1, df2 = 2070, p-value = 0.4257


# Summary of Period 3 models

In [89]:
bekk_c <- rep(NA,14)
for(i in 1:14){
    bekk_c[i] = paste0(round(bekkc[i,2],4)," (",round(bekkc[i,3],4),")")
}

bekk_1 <- rep(NA,14)
for(i in 1:14){
    bekk_1[i] = paste0(round(bekk1[i,2],4)," (",round(bekk1[i,3],4),")")
}

bekk_2 <- rep(NA,14)
for(i in 1:14){
    bekk_2[i] = paste0(round(bekk2[i,2],4)," (",round(bekk2[i,3],4),")")
}

bekk_3 <- rep(NA,14)
for(i in 1:14){
    bekk_3[i] = paste0(round(bekk3[i,2],4)," (",round(bekk3[i,3],4),")")
}

bekk_4 <- rep(NA,14)
for(i in 1:14){
    bekk_4[i] = paste0(round(bekk4[i,2],4)," (",round(bekk4[i,3],4),")")
}

bekk_5 <- rep(NA,14)
for(i in 1:14){
    bekk_5[i] = paste0(round(bekk5[i,2],4)," (",round(bekk5[i,3],4),")")
}

bekk_6 <- rep(NA,14)
for(i in 1:14){
    bekk_6[i] = paste0(round(bekk6[i,2],4)," (",round(bekk6[i,3],4),")")
}

bekk_7 <- rep(NA,14)
for(i in 1:14){
    bekk_7[i] = paste0(round(bekk7[i,2],4)," (",round(bekk7[i,3],4),")")
}


bekk_8 <- rep(NA,14)
for(i in 1:14){
    bekk_8[i] = paste0(round(bekk8[i,2],4)," (",round(bekk8[i,3],4),")")
}

In [90]:
parameters1 = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22")
daily_bekk1 <- data.frame("Parameters" = parameters1,"BEKKC" = bekk_c,"BEKK1" = bekk_1,"BEKK2" = bekk_2,"BEKK3" = bekk_3,"BEKK4" = bekk_4,"BEKK5" = bekk_5,
                         "BEKK6" = bekk_6,"BEKK7" = bekk_7,"BEKK8" = bekk_8)
daily_bekk1


Parameters,BEKKC,BEKK1,BEKK2,BEKK3,BEKK4,BEKK5,BEKK6,BEKK7,BEKK8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,-2e-04 (5e-04),-2e-04 (5e-04),-2e-04 (5e-04),-2e-04 (5e-04),-1e-04 (4e-04),-1e-04 (4e-04),-1e-04 (4e-04),-2e-04 (5e-04),-1e-04 (4e-04)
mu20,-1e-04 (4e-04),0 (1e-04),-1e-04 (2e-04),1e-04 (1e-04),1e-04 (2e-04),2e-04 (2e-04),2e-04 (2e-04),-4e-04 (3e-04),-2e-04 (3e-04)
phi11,-0.0683 (0.0647),-0.0379 (0.0314),-0.0405 (0.0317),-0.0362 (0.0311),-0.0108 (0.0312),-0.0116 (0.0313),-0.0106 (0.0312),-0.0311 (0.0311),-0.0088 (0.0311)
phi12,0.0507 (0.077),0.1748 (0.1179),0.1249 (0.0851),0.2497 (0.1215),0.0838 (0.0703),0.1001 (0.0823),0.0933 (0.074),0.0175 (0.057),0.0313 (0.0392)
phi21,-0.037 (0.0543),0.0148 (0.0084),0.0038 (0.0118),0.0038 (0.008),-0.0019 (0.0139),0.0124 (0.0119),0.0017 (0.0131),0.0279 (0.0157),0.0292 (0.0238)
phi22,0.0314 (0.0647),-0.0524 (0.0314),0.0168 (0.0318),0.0027 (0.0312),0.0182 (0.0312),0.0317 (0.0312),0.0279 (0.0312),-0.3821 (0.0287),-0.2721 (0.03)
a11,0.1091 (0.0816),0.1486 (0.0228),0.1268 (0.0296),0.1811 (0.0393),0.1701 (0.0305),0.1718 (0.0323),0.1641 (0.0274),0.1109 (0.0243),0.1618 (0.0303)
a12,-0.1915 (0.0704),0.0368 (0.0109),-0.0121 (0.0138),0.0283 (0.0159),1e-04 (0.0171),0.005 (0.0145),0.0253 (0.0155),-0.0178 (0.0099),0.0809 (0.0203)
a21,0.0797 (0.0898),0.4707 (0.0783),0.1723 (0.0779),0.4813 (0.1545),0.0251 (0.0817),0.0404 (0.0905),0.0305 (0.0761),-0.0339 (0.0423),-0.0162 (0.0353)


# Final VAR-BEKK Table for Period 2 (Table A.4)

In [91]:
daily_bekk <- data.frame("Parameters" = parameters1,
                         "BEKKC"=bekk_c,
                         "BEKK1"=bekk_1,
                         "BEKK2"=bekk_2,
                          "BEKK3"=bekk_3,
                          "BEKK4"=bekk_4,
                         "BEKK5"=bekk_5,
                          "BEKK6"=bekk_6,
                         "BEKK7" = bekk_7,
                         "BEKK8" = bekk_8)
daily_bekk

Parameters,BEKKC,BEKK1,BEKK2,BEKK3,BEKK4,BEKK5,BEKK6,BEKK7,BEKK8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,-2e-04 (5e-04),-2e-04 (5e-04),-2e-04 (5e-04),-2e-04 (5e-04),-1e-04 (4e-04),-1e-04 (4e-04),-1e-04 (4e-04),-2e-04 (5e-04),-1e-04 (4e-04)
mu20,-1e-04 (4e-04),0 (1e-04),-1e-04 (2e-04),1e-04 (1e-04),1e-04 (2e-04),2e-04 (2e-04),2e-04 (2e-04),-4e-04 (3e-04),-2e-04 (3e-04)
phi11,-0.0683 (0.0647),-0.0379 (0.0314),-0.0405 (0.0317),-0.0362 (0.0311),-0.0108 (0.0312),-0.0116 (0.0313),-0.0106 (0.0312),-0.0311 (0.0311),-0.0088 (0.0311)
phi12,0.0507 (0.077),0.1748 (0.1179),0.1249 (0.0851),0.2497 (0.1215),0.0838 (0.0703),0.1001 (0.0823),0.0933 (0.074),0.0175 (0.057),0.0313 (0.0392)
phi21,-0.037 (0.0543),0.0148 (0.0084),0.0038 (0.0118),0.0038 (0.008),-0.0019 (0.0139),0.0124 (0.0119),0.0017 (0.0131),0.0279 (0.0157),0.0292 (0.0238)
phi22,0.0314 (0.0647),-0.0524 (0.0314),0.0168 (0.0318),0.0027 (0.0312),0.0182 (0.0312),0.0317 (0.0312),0.0279 (0.0312),-0.3821 (0.0287),-0.2721 (0.03)
a11,0.1091 (0.0816),0.1486 (0.0228),0.1268 (0.0296),0.1811 (0.0393),0.1701 (0.0305),0.1718 (0.0323),0.1641 (0.0274),0.1109 (0.0243),0.1618 (0.0303)
a12,-0.1915 (0.0704),0.0368 (0.0109),-0.0121 (0.0138),0.0283 (0.0159),1e-04 (0.0171),0.005 (0.0145),0.0253 (0.0155),-0.0178 (0.0099),0.0809 (0.0203)
a21,0.0797 (0.0898),0.4707 (0.0783),0.1723 (0.0779),0.4813 (0.1545),0.0251 (0.0817),0.0404 (0.0905),0.0305 (0.0761),-0.0339 (0.0423),-0.0162 (0.0353)
